In [18]:
import requests, json
from pprint import pprint

In [19]:
base_url = 'https://fantasy.premierleague.com/api/'

#get data from bootstrap-static endpoint
r = requests.get(base_url+'bootstrap-static/').json()

#show the top-level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 9034127}


In [20]:
#get player data from 'elements' field
players = r['elements']

#show data for first player
pprint(players[0])

{'assists': 0,
 'bonus': 0,
 'bps': 48,
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'clean_sheets': 0,
 'code': 80201,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -5,
 'cost_change_start_fall': 5,
 'creativity': '0.0',
 'creativity_rank': 600,
 'creativity_rank_type': 69,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 1,
 'ep_next': '0.0',
 'ep_this': '1.0',
 'event_points': 0,
 'first_name': 'Bernd',
 'form': '0.0',
 'goals_conceded': 9,
 'goals_scored': 0,
 'ict_index': '7.9',
 'ict_index_rank': 425,
 'ict_index_rank_type': 30,
 'id': 1,
 'in_dreamteam': False,
 'influence': '79.0',
 'influence_rank': 367,
 'influence_rank_type': 29,
 'minutes': 270,
 'news': '',
 'news_added': '2022-02-11T08:00:15.144286Z',
 'now_cost': 45,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalti

In [21]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [22]:
#create players dataframe
players = pd.json_normalize(r['elements'])

#show some information about first five players
players[['id','web_name','team','element_type']].head()

,id,web_name,team,element_type
0,1,Leno,1,1
1,2,Rúnarsson,1,1
2,3,Willian,1,3
3,4,Aubameyang,1,4
4,5,Cédric,1,2


In [23]:
#create teams dataframe
teams = pd.json_normalize(r['teams'])

teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,4,None,False,0,1210,1270,1150,1210,1190,1220,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,3,None,False,0,1130,1160,1140,1110,1090,1090,2
2,94,0,None,3,0,Brentford,0,0,0,BRE,2,None,False,0,1060,1070,1120,1150,1080,1120,130
3,36,0,None,4,0,Brighton,0,0,0,BHA,3,None,False,0,1130,1130,1160,1160,1100,1120,131
4,90,0,None,5,0,Burnley,0,0,0,BUR,2,None,False,0,1060,1050,1080,1130,1060,1100,43


In [24]:
#get position information from 'element_types' field
positions = pd.json_normalize(r['element_types'])

positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],83
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],242
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],296
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],93


In [25]:
#join players to teams
df = pd.merge(
    left=players,
    right=teams,
    left_on='team',
    right_on='id'
)

#show joined result
df[['first_name','second_name','name']].head()

,first_name,second_name,name
0,Bernd,Leno,Arsenal
1,Rúnar Alex,Rúnarsson,Arsenal
2,Willian,Borges Da Silva,Arsenal
3,Pierre-Emerick,Aubameyang,Arsenal
4,Cédric,Soares,Arsenal


In [26]:
#join player positions
df = df.merge(
    positions,
    left_on='element_type',
    right_on='id'
)

#rename columns
df = df.rename(
    columns={'name':'team_name','singular_name':'position_name'}
)

#show result
df [
    ['first_name','second_name','team_name','position_name']
].head()

,first_name,second_name,team_name,position_name
0,Bernd,Leno,Arsenal,Goalkeeper
1,Rúnar Alex,Rúnarsson,Arsenal,Goalkeeper
2,Karl,Hein,Arsenal,Goalkeeper
3,Aaron,Ramsdale,Arsenal,Goalkeeper
4,Arthur,Okonkwo,Arsenal,Goalkeeper


In [27]:
#get data from 'element-summary/{PID}/' endpoint for PID=4
r = requests.get(base_url + 'element-summary/4/').json()

#show top level fields for player summary
pprint(r, depth=1)

#show data for 25th gameweek
pprint(r['history'][24])


{'fixtures': [...], 'history': [...], 'history_past': [...]}
{'assists': 0,
 'bonus': 0,
 'bps': 0,
 'clean_sheets': 0,
 'creativity': '0.0',
 'element': 4,
 'fixture': 279,
 'goals_conceded': 0,
 'goals_scored': 0,
 'ict_index': '0.0',
 'influence': '0.0',
 'kickoff_time': '2022-03-06T14:00:00Z',
 'minutes': 0,
 'opponent_team': 18,
 'own_goals': 0,
 'penalties_missed': 0,
 'penalties_saved': 0,
 'red_cards': 0,
 'round': 28,
 'saves': 0,
 'selected': 118640,
 'team_a_score': 3,
 'team_h_score': 2,
 'threat': '0.0',
 'total_points': 0,
 'transfers_balance': -1395,
 'transfers_in': 0,
 'transfers_out': 1395,
 'value': 96,
 'was_home': False,
 'yellow_cards': 0}


In [28]:
def get_gameweek_history(player_id):
    '''get all gameweek info for a given player_id'''

    #send GET request to
    #https://fantasy.premierleague.com/api/element-summary/{PID}/

    r = requests.get(
        base_url + 'element-summary/'+ str(player_id) + '/'
    ).json()

    #extract 'history' data from response into dataframe
    df = pd.json_normalize(r['history'])

    return df

#show player #4's gameweek history
get_gameweek_history(4)[
    [
        'round',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head()


,round,total_points,minutes,goals_scored,assists
0,1,0,0,0,0
1,2,1,29,0,0
2,3,1,58,0,0
3,4,9,90,1,0
4,5,2,90,0,0


In [29]:
def get_season_history(player_id):
    ''''get all past season_history for a given player_id'''

    #send GET request to
    #https://fantasy.premierleague.com/api/element-summary/{PID}/
    r =requests.get(
        base_url + 'element-summary/' + str(player_id) + '/'
    ).json()

    #extract 'history_past' data from response into dataframe
    df = pd.json_normalize(r['history_past']) 

    return df

#show player #1's gameweek history
get_season_history(1)[
    [
        'season_name',
        'total_points',
        'minutes',
        'goals_scored',
        'assists'
    ]
].head(10)



,season_name,total_points,minutes,goals_scored,assists
0,2018/19,106,2835,0,0
1,2019/20,114,2649,0,0
2,2020/21,131,3131,0,0


In [30]:
# select columns of interest from players df
players = players[
    ['id', 'first_name', 'second_name', 'web_name', 'team',
     'element_type']
]

# join team name
players = players.merge(
    teams[['id', 'name']],
    left_on='team',
    right_on='id',
    suffixes=['_player', None]
).drop(['team', 'id'], axis=1)

# join player positions
players = players.merge(
    positions[['id', 'singular_name_short']],
    left_on='element_type',
    right_on='id'
).drop(['element_type', 'id'], axis=1)

players.head()

,id_player,first_name,second_name,web_name,name,singular_name_short
0,1,Bernd,Leno,Leno,Arsenal,GKP
1,2,Rúnar Alex,Rúnarsson,Rúnarsson,Arsenal,GKP
2,532,Karl,Hein,Hein,Arsenal,GKP
3,559,Aaron,Ramsdale,Ramsdale,Arsenal,GKP
4,572,Arthur,Okonkwo,Okonkwo,Arsenal,GKP


In [31]:
from tqdm.auto import tqdm
tqdm.pandas()



In [32]:
# get gameweek histories for each player
points = players['id_player'].progress_apply(get_gameweek_history)

# combine results into single dataframe
points = pd.concat(df for df in points)

# join web_name
points = players[['id_player', 'web_name']].merge(
    points,
    left_on='id_player',
    right_on='element'
)


100%|██████████| 714/714 [07:10<00:00,  1.66it/s]  


In [33]:
#get top scoring players
points.groupby(
    ['element','web_name']
).agg(
    {'total_points':'sum','goals_scored':'sum','assists':'sum'}
).reset_index(
).sort_values(
    'total_points',ascending=False
).head()

,element,web_name,total_points,goals_scored,assists
232,233,Salah,212,19,11
236,237,Alexander-Arnold,174,2,11
419,420,Bowen,153,8,13
255,256,Cancelo,153,1,8
358,359,Son,138,10,7
